# Correlation Between Semantic Similarity and Wikipedia Distance
### CMSC 320, Spring 2022
#### Ben Landrum, Gaetan Almela, and Nav Bindra



The Wikipedia link graph is a graph where vertices represent Wikipedia articles and directed edges from vertex $A$ to vertex $B$ represent a link to $B$ in the text of $A$. We define Wikipedia distance between pages $A$ and $B$ to be the minimum number of links required to get from $A$ to $B$ in the Wikipedia link graph, where the first such link is on page $A$, the second is on the page the first link points to, and so on, until a link to $B$ is found.

Word2Vec is a popular tool for computing word embeddings, which use a vector space representation of words to place them in an abstract space where similar vectors indicate that words appear in similar contexts. These vectors can then be compared to find a metric of semantic similarity between words. 

Because pages on Wikipedia contain links which are relevant to the topic the page is covering, similar topics are often closely if not directly linked. Therefore, it stands to reason that if semantic similarity is a valid metric of similarity between topics, a higher semantic similarity between two topics should correspond to a lower Wikipedia distance between said topics. We are looking to test the hypothesis that there exists a negative correlation between Wikipedia distance and Word2Vec semantic similarity. Our null hypothesis is that there does not exist a statistically significant negative correlation between Wikipedia distance and Word2Vec semantic similarity.

In [2]:
from bs4 import BeautifulSoup
from requests import get
from faker import Faker

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import spearmanr

import time
import re
import os
from pathlib import Path
import datetime
import warnings

from gensim.models import KeyedVectors

## Computing Wikipedia Distance

### Top $n$ Wikipedia Articles

English Wikipedia has over 6 million articles. A graph describing the links between all of them would be prohibitively large, so we will only consider the top $n$ articles for some large $n$. For $n > 1,000$ this is not something that can be found online, so we must compute it ourselves.

Wikipedia provides data dumps describing page views in the form of files tallying how many views each page received in a given hour. These files are available at [https://dumps.wikimedia.org/other/pageviews/](https://dumps.wikimedia.org/other/pageviews/), and date back to May 1st, 2015. The most popular pages on Wikipedia at any given time are generally determined by current events, so to factor out this bias and get a representative sample of the most popular articles we want to sample the largest possible time period, in this case May 1st, 2015 to the present day.

In [3]:
ENGLISH_WIKIS = ['en', 'en.m'] # desktop and mobile versions of english wikipedia
PAGE_TYPE_REGEX = '^((User)|(Talk)|(Wikipedia)|(Special)|(Portal)|(File)):' # regex for filtering non-article pages
N = 10 ** 4 # number of pages to return
OUTPUT_FILE = 'wiki_top_k.csv' # name of output file
DATA_DIR = 'Data' # directory where the data is stored
RANDOM_FILES = 0 # Number of random files to use
CACHED_PAGEVIEWS = 'Data/raw_views_5M.csv.gz' # name of cached pageviews file

# creating a Faker object which will later be used to generate random datetimes
fake = Faker()

# suppressing a warning caused by use of match groups in a regex 
warnings.filterwarnings('ignore', message="This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.")
warnings.filterwarnings('ignore', message="This pattern has match groups. To actually get the groups, use str.extract.")

In [4]:
if RANDOM_FILES > 0:
    # generate a random datetime for which there exists a pageview file
    date = fake.date_time_between(start_date=datetime.datetime(year=2015, month=5, day=1, hour=1), end_date='now')

    # get the url of the corresponding pageview file
    url = f"https://dumps.wikimedia.org/other/pageviews/{date.year}/{date.year}-{date.month:02d}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz"

    # download and save the file
    seed_file = f"{DATA_DIR}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz"
    r = get(url)
    with open(seed_file, 'wb') as f:
        f.write(r.content)

    # importing first file to accumulate the others from
    all_views = pd.read_csv(seed_file, sep=" ", header=None, names=["project", "page", "views", "null"], usecols=["project", "page", "views"])

    # filtering to only pages from english wikipedia, both desktop and mobile
    en_views = all_views[all_views['project'].isin(ENGLISH_WIKIS)]
    # filtering user, talk and other non-article pages
    views = en_views[~(en_views['page'].str.contains(PAGE_TYPE_REGEX, na=False))]

    # computing views for each page 
    views = views.groupby('page').sum()

/Users/ben/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Now we get the random files and add their views to the seed file, which becomes increasingly expensive as `all_views` grows.

In [5]:
start_time = datetime.datetime.now() # for timing execution
for i in range(max(RANDOM_FILES - 1, 0)):
    # generate a random datetime for which there exists a pageview file
    date = fake.date_time_between(start_date=datetime.datetime(year=2015, month=5, day=1, hour=1), end_date='now')

    # get the url of the corresponding pageview file
    url = f"https://dumps.wikimedia.org/other/pageviews/{date.year}/{date.year}-{date.month:02d}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz"

    # download and save the file
    r = get(url)
    with open(f"{DATA_DIR }/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz", 'wb') as f:
        f.write(r.content)
    print(f"{date.strftime('%Y/%m/%d:%H')}")
    
    # read the file (pandas can handle gzip directly) and perform same operations as above
    try:
        df = pd.read_csv(f"{DATA_DIR}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz", compression='gzip', sep=" ", header=None, names=["project", "page", "views", "null"], usecols=["project", "page", "views"])
    except: # handling error caused by pandas' "C" CSV engine
        print(f"{date.strftime('%Y/%m/%d:%H')} failed" )
        os.remove(f"{DATA_DIR}/pageviews-{date.strftime('%Y%m%d')}-{date.hour:02d}0000.gz")
        continue

    df = df[df['project'].isin(ENGLISH_WIKIS)]
    df = df[~(df['page'].str.contains(PAGE_TYPE_REGEX, na=False))]
    df = df.groupby('page').sum()
    # merging with previous dataframe
    views = views.add(df, fill_value=0)

    # print status report
    print(f"[{i + 1}]\t{date.strftime('%Y/%m/%d:%H')}\t{len(views):,} pages\t[{datetime.datetime.now() - start_time} elapsed, {(i * 100) / RANDOM_FILES:.2f}% complete, {(datetime.timedelta(seconds=((datetime.datetime.now() - start_time).total_seconds() / (i + 1)) * (RANDOM_FILES - i - 1)))} remaining]")

2017/09/11:14
[1]	2017/09/11:14	2,289,812 pages	[0:00:30.764682 elapsed, 0.00% complete, 0:02:33.823470 remaining]
2017/03/20:18
[2]	2017/03/20:18	2,950,995 pages	[0:01:02.051994 elapsed, 16.67% complete, 0:02:04.104022 remaining]
2020/05/24:20
[3]	2020/05/24:20	3,509,750 pages	[0:01:36.872728 elapsed, 33.33% complete, 0:01:36.872741 remaining]
2019/11/18:09
[4]	2019/11/18:09	3,834,938 pages	[0:02:26.075450 elapsed, 50.00% complete, 0:01:13.037732 remaining]
2017/07/16:01
[5]	2017/07/16:01	4,078,113 pages	[0:02:59.625499 elapsed, 66.67% complete, 0:00:35.925102 remaining]


Now we save the pages and their views to a csv.

In [6]:
views.sort_values('views', ascending=False, inplace=True)

views.head(N).to_csv(OUTPUT_FILE, index=True)

A more optimized version of this code was run over 12 hours, sampling a total of 1,105 hours of data. The top 5 million articles from this dataset are in the file `raw_views_5M.csv`, and we'll use them for the rest of this project.

Before we start scraping Wikipedia, we have to define some constants for our scraper. Scraping Wikipedia is suprisingly easy, the DOM tree and class names are very consistent. Despite this, Wikipedia still has a lot of garbage links that we don't want. This is where we define all of our rules for what counts as a "bad" link.  Examples of bad links include images, self links, internal links (links to enlarge photos, believe it or not), or invisible links that take you to the top of the page.

One might think to simply filter by links containing `/wiki/`, but even those links aren't safe. Indeed Wikipedia contains various different types of internal `/wiki/` links that point to various medias, template pages, or help files, which we are not interested in. We also filter them here.

In [7]:
DATA_DIR = 'data'
GRAPH_FILENAME = 'wiki_graph'

# number of pages to experiment on
N = 1000

# we filter out these types of link selectors
CLEAN = [
  'a[id="top"]',
  'a[class="mw-selflink selflink"]',
  'a[class="image"]',
  'a[class="internal"]',
]

# we filter out these links
REM_LINKS = [
  r"(\/wiki\/File:\w+)",
  r"(\/wiki\/Special:\w+)",
  r"(\/wiki\/Template:\w+)",
  r"(\/wiki\/Category:\w+)",
  r"(\/wiki\/Portal:\w+)",
  r"(\/wiki\/Template_talk:\w+)",
  r"(\/wiki\/Help:\w+)",
  r"(\/wiki\/Wikipedia:\w+)",
  r"(^#\w+)",
]

# main page content selector
CONT_SEL = "div#content"

Here, we load the list of pages that we are interested in scraping

In [8]:
# load most popular wikipedia pages csv
df = pd.read_csv(CACHED_PAGEVIEWS).head(N)

# get a list of pages as an array of strings
urls = df['article'].to_numpy().astype(str)

# filter the pages to only articles without ':' in the title
urls = urls[np.char.find(urls, ':') == -1]

FileNotFoundError: [Errno 2] No such file or directory: 'Data/raw_views_5M.csv.gz'

## Download the Pages Locally

To avoid overwhelming Wikipedia we download every page, unmodified, locally. This allows us to experiment with parsing the Wikipedia pages without having to redownload the pages every run. This is also much faster since we are working with local files. The reason we leave the files unmodified here instead of cleaning them at the same time is that we always want to have the maximum amount of information possible first (in case you need it) and clean it up as a separate step.

One thing to note here is that we also add a variable `START_INDEX` to manually start from a particular index in our list of links to scrape. This serves two purposes. The first is in case a special page break our code, we don't have to restart from the beginning to rescrape the pages we already have. The second is for this page not to take 50 hours to compile to HTML.

In [ ]:
# in case you wanna skip ahead
START_INDEX = 970
SLEEP_TIME_S = 2 # dont make less than 2, we dont wanna overwhelm wikipedia too much

n = urls.size

# compile the title cleanup regex for optimization
title_re = re.compile(r"\/")

for i, url in enumerate(urls):
  if i < START_INDEX:
    continue

  print(f"{i} of {n} ({round((i / n) * 100, 2)}%) - Scraping {url}")

  # load the page as html with BeautifulSoup
  res = get(f'https://en.wikipedia.org/wiki/{url}')

  # check if we got baned :c
  if res.status_code != 200:
    print("We got got")
    break

  html = BeautifulSoup(res.text, 'html.parser')

  # save html page as string
  html_str = str(html.prettify())

  # replace bad characters in titles with underscores
  title = title_re.sub("_", url)

  # save file
  f = open(f"{DATA_DIR}/pages/{title}.html", "w")
  f.write(html_str)
  f.close()

  # let's not overload wikipedia with requests here
  time.sleep(SLEEP_TIME_S)

## Building the Graph

Here we use our local data and our filters defined above to build our page connections graph with `networkx`. We iterate through each page in our local file structure and look at each link that matches our filters defined in our constants above. Note that we don't add every url that passes our filter here, _we only add an edge if its destination page is also in our list of pages_. This helps keep the size of our graph reasonable.

In [ ]:
files = os.scandir(path = f"./{DATA_DIR}/pages/")
n = len(list(files))

In [ ]:
G = nx.DiGraph()

# list our downloaded files
files = os.scandir(path = f"./{DATA_DIR}/pages/")
# n = len(list(files))

for i, file in enumerate(files):
  # extract url from filename
  url = Path(file).stem

  print(f"{i} of {n} ({round((i / n) * 100, 2)}%) - Graphing {url}")

  # add the current page to the graph
  G.add_node(url)

  # load the page into memory
  f = open(file, "r")
  html = BeautifulSoup(f.read(), 'html.parser')
  f.close()

  # clean up unwanted elements from page
  for c in CLEAN:
    els = html.select(f"{CONT_SEL} {c}")
    for el in els:
      el.decompose()

  # further remove unwanted links
  links = html.select(f"{CONT_SEL} a")
  for link in links:
    # extract href from link
    href = link['href']

    # we are only interested in links that start with `/wiki/` and aren't in REM_LINKS
    if not any([re.match(regex, href) for regex in REM_LINKS]) and re.match(r"^\/wiki\/\w+", href):
      # extract href from link
      href = link['href']

      # this is the destination url
      url_ = re.sub(r"\/wiki\/", "", href)

      # we only add the destination url if it happens to be inside our links of interests
      if url_ in urls:
        # add edge to graph
        G.add_edge(url, url_)

  # save our graph at other file
  if i % 3 == 0:
    nx.write_gml(G, f"{DATA_DIR}/{GRAPH_FILENAME}.gml")

Here, we draw the graph of the top 1000 pages.

In [ ]:
nodes = np.array(G.nodes)

sub_nodes = np.random.choice(nodes, size = 25)
sub_nodes

fig, ax = plt.subplots(figsize=(20, 20))

nodes = (G.nodes)
N_nodes = G.number_of_nodes()

pos = nx.circular_layout(G)

# Setting up the text by using node position 
texts=[
  plt.text(
    pos[node][0], 
    pos[node][1],
    str(node),
    rotation=(i/N_nodes)*360,
    fontsize=10,horizontalalignment='center',
    verticalalignment='center'
  )
  
  for i, node in enumerate(nodes)
]

nx.draw(G, pos=pos, arrows=True, node_size=50, linewidths=0.6, ax=ax)
ax.set_aspect('equal')
plt.show()

## Computing Semantic Similarity with Word2Vec

We now use `gensim` to load a Word2Vec embedding. The embedding used is `crawl-300d-2M.vec`, two million word vectors trained on the 600 billion token Common Crawl corpus. It can be downloaded from https://fasttext.cc/docs/en/english-vectors.html. As previously discussed, the embedding represents words in a 300 dimensional vector space, where the inner product of the vectors of two words corresponds to how commonly they occur in the same documents. 

Because the Common Crawl is a set of webpages, Wikipedia pages are likely included in the sample. As a result, the presence of a link on a Wikipedia page to another Wikipedia page would cause the titles to be more closely associated in the embedding, because the link and title being in the same page would represent a co-occurrence of the two terms or sets of terms. However, because Wikipedia represents a vanishingly small fraction of the web, the direct influence of this is minimal, and we will disregard it. 

In [ ]:
MODEL_BIN = 'data/model/crawl-300d-2M.vec'
save_name = MODEL_BIN[:-4] + ".d2v"

# loading model
model = KeyedVectors.load(save_name)

# get words included in model
model_keys = model.index_to_key

We now define a function that will return the semantic similarity of two titles.

In [ ]:
def semanticSimilarity(w1, w2):
	w1_tokens = w1.split('_') # split into tokens
	w1_tokens = list(filter(lambda x: x in model_keys, w1_tokens)) # exclude terms not in embedding

	w2_tokens = w2.split('_') 
	w2_tokens = list(filter(lambda x: x in model_keys, w2_tokens))

	return model.n_similarity(w1_tokens, w2_tokens)

## Hypothesis Test

We now proceed to test our hypothesis that the distance between a pair of articles on the Wikipedia graph is correlated with the semantic similarity between the titles of the articles. 

A slight issue is presented by the fact that Wikipedia distance is directional, whereas the semantic similarity is not. As a result, it's possible that the distance from article $A$ to article $B$ may differ from the distance from article $B$ to article $A$. Because the direction of a pair of articles is sampled randomly, there is no bias introduced by this unidirectional sample, so it's acceptable to compare only one direction's distance to the semantic similarity.

Generate k article pairs from top n:

In [ ]:
k = 5000
k_pairs = pd.DataFrame(columns=['start','end','wikiDistance','semanticSimilarity'])

# G = nx.read_gml('data/n_wiki_graph.gml') # if the graph isn't already loaded

for i in range(k):
	temp = urls.sample(n=2, random_state=i)
	start = temp.values[0]
	end = temp.values[1]

	if start not in list(G.nodes) or end not in list(G.nodes):
		continue
	# compute distance
	if nx.has_path(G, start, end):
		dist = len(nx.shortest_path(G, start, end)) - 1
	else:
		dist = -1

	# compute semantic similarity
	semSim = semanticSimilarity(start, end)

	k_pairs = pd.concat(k_pairs, {'start':start, 'end':end, 'wikiDistance':dist, 'semanticSimilarity':semSim}, ignore_index=True)

k_pairs.head()

We now use Spearman's r test to compute the probability that there exists a statistically significant correlation between Wikipedia distance and semantic similarity.

In [ ]:
corr, p_val = spearmanr(k_pairs['wikiDistance'], k_pairs['semanticSimilarity'])
print(f"p-value: {p_val}")

This correlation can be visualized on a violin plot of the semantic similarities of article pairs at each observed distance.

In [ ]:
distances = [i for i in range(1, 8)]
similarities = [k_pairs[k_pairs['wikiDistance'] == i]['semanticSimilarity'].array for i in distances]

plt.violinplot(similarities, positions=distances, showmeans=True, showextrema=True)
plt.xticks(distances, distances)
plt.xlabel('Wiki Distance')
plt.ylabel('Semantic Similarity')
plt.title('Semantic Similarity vs. Wiki Distance')
plt.show()